In [1]:
import os 
from dotenv import load_dotenv
load_dotenv()

groq_api_key = os.getenv("GROQ_API_KEY")


In [2]:
from langchain_groq import ChatGroq
model = ChatGroq(model="Gemma2-9b-It",
groq_api_key=groq_api_key)
model

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x000001C399D172E0>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x000001C399D517B0>, model_name='Gemma2-9b-It', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [3]:
from langchain_core.messages import HumanMessage
model.invoke([HumanMessage(content="Hi, my name is Sarthak and I am a student")])


AIMessage(content="Hi Sarthak, it's nice to meet you!\n\nWhat can I do for you today? Are you working on a school project, looking for information, or just want to chat?  I'm happy to help in any way I can.\n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 56, 'prompt_tokens': 22, 'total_tokens': 78, 'completion_time': 0.101818182, 'prompt_time': 0.001641808, 'queue_time': 0.25975258199999995, 'total_time': 0.10345999}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None}, id='run--3503b6bb-0da8-4875-8e8b-37f107a0a288-0', usage_metadata={'input_tokens': 22, 'output_tokens': 56, 'total_tokens': 78})

In [5]:
from langchain_core.messages import AIMessage
model.invoke(
    [
        HumanMessage(content="Hi , My name is Sarthak and I am a student"),
        AIMessage(content="Hi Sarthak, it's nice to meet you!\n\nWhat can I do for you today? Are you working on a school project, looking for information, or just want to chat?  I'm happy to help in any way I can.\n"),
        HumanMessage(content="Hey What's my name and what do I do?")
    ]
)

AIMessage(content="You told me your name is Sarthak and that you are a student.  \n\nIs there anything else you'd like to tell me about yourself?  😊 \n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 38, 'prompt_tokens': 98, 'total_tokens': 136, 'completion_time': 0.069090909, 'prompt_time': 0.002664918, 'queue_time': 0.257925394, 'total_time': 0.071755827}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None}, id='run--6edd2b84-b345-4c2a-9c01-7f7ac3bf88b8-0', usage_metadata={'input_tokens': 98, 'output_tokens': 38, 'total_tokens': 136})

In [6]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

store={}

def get_session_history(session_id:str)->BaseChatMessageHistory:
    if session_id not in store:
        store[session_id]=ChatMessageHistory()
    return store[session_id]

with_message_history=RunnableWithMessageHistory(model,get_session_history)

In [7]:
config={"configurable":{"session_id":"chat1"}}

In [8]:
response=with_message_history.invoke(
    [HumanMessage(content="Hi , My name is Sarthak and I am a student")],
    config=config
)

In [9]:
response.content

'Hi Sarthak! Nice to meet you. \n\nWhat can I do for you today? Are you working on a school project?  Do you have any questions I can help you with?  😊  \n\n'

In [10]:
with_message_history.invoke(
    [HumanMessage(content="What's my name?")],
    config=config,
)

AIMessage(content='Your name is Sarthak. 😊  \n\nI remembered!  Is there anything else I can help you with?\n', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 27, 'prompt_tokens': 82, 'total_tokens': 109, 'completion_time': 0.049090909, 'prompt_time': 0.002451378, 'queue_time': 0.24970631599999998, 'total_time': 0.051542287}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None}, id='run--24062a50-2904-43e4-8b98-2dc176bd1e40-0', usage_metadata={'input_tokens': 82, 'output_tokens': 27, 'total_tokens': 109})

In [11]:
## change the config-->session id
config1={"configurable":{"session_id":"chat2"}}
response=with_message_history.invoke(
    [HumanMessage(content="Whats my name")],
    config=config1
)
response.content

"As an AI, I have no memory of past conversations and do not know your name. If you'd like to tell me, I'm happy to use it! 😊\n"

In [12]:
response=with_message_history.invoke(
    [HumanMessage(content="Hey My name is John")],
    config=config1
)
response.content

'Hi John! Nice to meet you. 😊\n\nWhat can I do for you today?\n'

In [13]:
response=with_message_history.invoke(
    [HumanMessage(content="Whats my name")],
    config=config1
)
response.content

'Your name is John, remember?  😄 \n\nDid you forget for a moment?  \n'

In [14]:
from langchain_core.prompts import ChatPromptTemplate,MessagesPlaceholder
prompt=ChatPromptTemplate.from_messages(
    [
        ("system","You are a helpful assistant.Answer all the question to the rest of your ability"),
        MessagesPlaceholder(variable_name="messages")
    ]
)

chain=prompt|model

In [15]:
chain.invoke({"messages":[HumanMessage(content="Hi My name is Sarthak")]})

AIMessage(content="Hello Sarthak, it's nice to meet you! \n\nI'm happy to help with any questions you have. Just ask away! 😊  \n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 36, 'prompt_tokens': 32, 'total_tokens': 68, 'completion_time': 0.065454545, 'prompt_time': 0.001480289, 'queue_time': 0.254657021, 'total_time': 0.066934834}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None}, id='run--3d8a503b-d51f-4281-b034-e775ff8f521e-0', usage_metadata={'input_tokens': 32, 'output_tokens': 36, 'total_tokens': 68})

In [16]:
with_message_history=RunnableWithMessageHistory(chain,get_session_history)

In [18]:
config = {"configurable": {"session_id": "chat3"}}
response=with_message_history.invoke(
    [HumanMessage(content="Hi My name is Sarthak")],
    config=config
)

response

AIMessage(content="Hi Sarthak! It's great to meet you too.  \n\nIs there anything I can help you with today?  \n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 30, 'prompt_tokens': 80, 'total_tokens': 110, 'completion_time': 0.054545455, 'prompt_time': 0.002495018, 'queue_time': 0.24975118300000002, 'total_time': 0.057040473}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None}, id='run--b7f2be6e-edd0-4416-9c7a-9df4120016d3-0', usage_metadata={'input_tokens': 80, 'output_tokens': 30, 'total_tokens': 110})

In [19]:
response = with_message_history.invoke(
    [HumanMessage(content="What's my name?")],
    config=config,
)

response.content

'Your name is Sarthak. 😊  \n\nIs there anything else I can help you with?\n'

In [20]:
## Add more complexity

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant. Answer all questions to the best of your ability in {language}.",
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

chain = prompt | model

In [21]:
response=chain.invoke({"messages":[HumanMessage(content="Hi My name is Sarthak")],"language":"Hindi"})
response.content

'नमस्ते Sarthak! 👋  मुझे खुशी है कि आप मुझसे बात कर रहे हैं।  😊 \n\nआप मुझसे कुछ भी पूछ सकते हैं, मैं अपनी पूरी कोशिश करूँगा कि आपको सटीक और मददगार जवाब दूं। \n'

In [22]:
with_message_history=RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="messages"
)

In [23]:
config = {"configurable": {"session_id": "chat4"}}
repsonse=with_message_history.invoke(
    {'messages': [HumanMessage(content="Hi,I am Sarthak")],"language":"Hindi"},
    config=config
)
repsonse.content

'नमस्ते सरथक ! 😊 \n\nमुझे आपकी मदद करने में खुशी हो रही है। क्या मैं आपकी कोई सहायता कर सकता हूँ? \n\n'

In [24]:
response = with_message_history.invoke(
    {"messages": [HumanMessage(content="whats my name?")], "language": "Hindi"},
    config=config,
)
response.content

'आपका नाम सरथक है। 😊  \n'

In [38]:
from langchain_core.messages import SystemMessage,trim_messages
trimmer=trim_messages(
    max_tokens=45,
    strategy="last",
    token_counter=model,
    include_system=True,
    allow_partial=False,
    start_on="human"
)
messages = [
    SystemMessage(content="you're a good assistant"),
    HumanMessage(content="hi! I'm bob"),
    AIMessage(content="hi!"),
    HumanMessage(content="I like vanilla ice cream"),
    AIMessage(content="nice"),
    HumanMessage(content="whats 2 + 2"),
    AIMessage(content="4"),
    HumanMessage(content="thanks"),
    AIMessage(content="no problem!"),
    HumanMessage(content="having fun?"),
    AIMessage(content="yes!"),
]
trimmer.invoke(messages)

[SystemMessage(content="you're a good assistant", additional_kwargs={}, response_metadata={}),
 HumanMessage(content='I like vanilla ice cream', additional_kwargs={}, response_metadata={}),
 AIMessage(content='nice', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='whats 2 + 2', additional_kwargs={}, response_metadata={}),
 AIMessage(content='4', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='thanks', additional_kwargs={}, response_metadata={}),
 AIMessage(content='no problem!', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='having fun?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='yes!', additional_kwargs={}, response_metadata={})]

In [44]:
from operator import itemgetter

from langchain_core.runnables import RunnablePassthrough

chain=(
    RunnablePassthrough.assign(messages=itemgetter("messages")|trimmer)
    | prompt
    | model
    
)

response=chain.invoke(
    {
    "messages":messages + [HumanMessage(content="What ice cream do i like")],
    "language":"English"
    }
)
response.content
trimmer.invoke(messages)

[SystemMessage(content="you're a good assistant", additional_kwargs={}, response_metadata={}),
 HumanMessage(content='I like vanilla ice cream', additional_kwargs={}, response_metadata={}),
 AIMessage(content='nice', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='whats 2 + 2', additional_kwargs={}, response_metadata={}),
 AIMessage(content='4', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='thanks', additional_kwargs={}, response_metadata={}),
 AIMessage(content='no problem!', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='having fun?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='yes!', additional_kwargs={}, response_metadata={})]

In [46]:
response = chain.invoke(
    {
        "messages": messages + [HumanMessage(content="what math problem did i ask")],
        "language": "English",
    }
)
response.content
trimmer.invoke(messages)

[SystemMessage(content="you're a good assistant", additional_kwargs={}, response_metadata={}),
 HumanMessage(content='I like vanilla ice cream', additional_kwargs={}, response_metadata={}),
 AIMessage(content='nice', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='whats 2 + 2', additional_kwargs={}, response_metadata={}),
 AIMessage(content='4', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='thanks', additional_kwargs={}, response_metadata={}),
 AIMessage(content='no problem!', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='having fun?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='yes!', additional_kwargs={}, response_metadata={})]

In [41]:
## Lets wrap this in the MEssage History
with_message_history = RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="messages",
)
config={"configurable":{"session_id":"chat5"}}

In [42]:
response = with_message_history.invoke(
    {
        "messages": messages + [HumanMessage(content="whats my name?")],
        "language": "English",
    },
    config=config,
)

response.content

"As a helpful assistant, I do not have access to any personal information about you, including your name.  If you'd like to tell me your name, I'd be happy to use it!\n"

In [43]:
response = with_message_history.invoke(
    {
        "messages": [HumanMessage(content="what math problem did i ask?")],
        "language": "English",
    },
    config=config,
)

response.content

"As an AI, I have no memory of past conversations.  \n\nCould you please tell me the math problem you'd like me to help you with? 😊  I'm ready! \n\n"